# CHAPTER 6
# Data Loading, Storage, and File Formats
- Accessing data is a necessary first step for using most of the tools in this book.
- Focus on data input and output using **pandas**.
- Reading text files and other more efficient on-disk formats.
- Loading data from databases.
- Interacting with network sources like web APIs.

## Reading and Writing Data in Text Format
- **pandas** features a number of functions for reading tabular data as a **DataFrame** object.
- Most common are **read_csv** and **read_table**.

**TABLE: Parsing functions in pandas**

| Function                  | Description |
| :---                  |    :----    |
|read_csv| Load delimited data from a file, URL, or file-like object; use comma as default delimiter
|read_table| Load delimited data from a file, URL, or file-like object; use tab ('\t') as default delimiter
|read_fwf| Read data in fixed-width column format (i.e., no delimiters)
|read_clipboard| Version of read_table that reads data from the clipboard; useful for converting tables from web pages
|read_excel| Read tabular data from an Excel XLS or XLSX file
|read_hdf| Read HDF5 files written by pandas
|read_html| Read all tables found in the given HTML document
|read_json| Read data from a JSON (JavaScript Object Notation) string representation
|read_msgpack| Read pandas data encoded using the MessagePack binary format
|read_pickle| Read an arbitrary object stored in Python pickle format
|read_sas| Read a SAS dataset stored in one of the SAS system’s custom storage formats
|read_sql| Read the results of a SQL query (using SQLAlchemy) as a pandas DataFrame
|read_stata| Read a dataset from Stata file format
|read_feather| Read the Feather binary file format

Optional arguments of these function:
- **Indexing**: 
        Can treat one or more columns as the returned DataFrame, and whether to get column names from the file, the user, or not at all.
- **Type inference and data conversion**:
        This includes the user-defined value conversions and custom list of missing value markers.
- **Datetime parsing**:
        Includes combining capability, including combining date and time information spread over multiple columns into a single column in the result.
- **Iterating**:
        Support for iterating over chunks of very large files.
- **Unclean data issues**:
        Skipping rows or a footer, comments, or other minor things like numeric data with thousands separated by commas.

- Because real world data is messy many data loading functions (especially read_csv) have grown very complex in their options over time.
- **read_csv** has over 50 parameters
- Some of these functions, like **pandas.read_csv**, perform **type inference**, because the column data types are not part of the data format. That means you don’t necessarily have to specify which columns are numeric, integer, boolean, or string.
- Other data formats, like HDF5, Feather, and msgpack, have the data types stored in the format.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np

In [ ]:
# Check the documetation for read_csv
pd.read_csv?

In [ ]:
# Use read_csv to read ex1.csv into a DataFrame
df = pd.read_csv('examples/ex1.csv')
df

In [ ]:
# One can specify the delimeter with read_csv
pd.read_table('examples/ex1.csv', sep=',')

In [ ]:
# Read file without header
pd.read_csv('examples/ex2.csv', header=None)

In [ ]:
# Specify the columns names
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

In [ ]:
# Specify that the 4th column is supposed to be the index of the DataFrame
pd.read_csv('examples/ex2.csv',
            names=['a', 'b', 'c', 'd', 'message'], index_col=4)

In [ ]:
# Create a hierarchical index from multiple columns
pd.read_csv('examples/csv_mindex.csv',
            index_col=['key1', 'key2'])

In [ ]:
# Consider a text file that does not have a fixed delimiter
# Fields here are separated by a variable amount of whitespace
list(open('examples/ex3.txt'))

In [ ]:
# Pass a regular expression as the delimiter
pd.read_table('examples/ex3.txt', sep='\s+')

# Because there was one fewer column name than the number of data rows,
# read_table infers that the first column should be the DataFrame’s index in this special case.

In [ ]:
# You can skip the first, third, and fourth rows of a file with skiprows
list(open('examples/ex4.csv'))

In [ ]:
# Read above file with read_csv
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

- Handling missing values is an important and frequently nuanced part of the file parsing process. 
- Missing data is usually either not present (empty string) or marked by some sentinel value. 
- By default, pandas uses a set of commonly occurring sentinels, such as **NA** and **NULL**.

In [ ]:
# Check file with missing values
list(open('examples/ex5.csv'))

In [ ]:
# Read file above using read_csv
result = pd.read_csv('examples/ex5.csv')
result

In [ ]:
# Check for missing values
pd.isnull(result)

**TABLE: Some read_csv/read_table function arguments**

| Argument                  | Description |
| :---                  |    :----    |
|path| String indicating filesystem location, URL, or file-like object
|sep or delimiter| Character sequence or regular expression to use to split fields in each row
|header| Row number to use as column names; defaults to 0 (first row), but should be None if there is no header row
|index_col| Column numbers or names to use as the row index in the result; can be a single name/number or a list of them for a hierarchical index
|names| List of column names for result, combine with header=None
|skiprows| Number of rows at beginning of file to ignore or list of row numbers (starting from 0) to skip.
|na_values| Sequence of values to replace with NA.
|comment| Character(s) to split comments off the end of lines.
|parse_dates| Attempt to parse data to datetime; False by default. If True, will attempt to parse all columns. Otherwise can specify a list of column numbers or name to parse. If element of list is tuple or list, will combine multiple columns together and parse to date (e.g., if date/time split across two columns).
|keep_date_col| If joining columns to parse date, keep the joined columns; False by default.
|converters| Dict containing column number of name mapping to functions (e.g., {'foo': f} would apply the function f to all values in the 'foo' column).
|dayfirst| When parsing potentially ambiguous dates, treat as international format (e.g., 7/6/2012 -> June 7, 2012); False by default.
|date_parser| Function to use to parse dates.
|nrows| Number of rows to read from beginning of file.
|iterator| Return a TextParser object for reading file piecemeal.
|chunksize| For iteration, size of file chunks.
|skip_footer| Number of lines to ignore at end of file.
|verbose| Print various parser output information, like the number of missing values placed in non-numeric columns.
|encoding| Text encoding for Unicode (e.g., 'utf-8' for UTF-8 encoded text).
|squeeze| If the parsed data only contains one column, return a Series.
|thousands| Separator for thousands (e.g., ',' or '.').

### Reading Text Files in Pieces
- When processing very large files or figuring out the right set of arguments to correctly process a large file, you may only want to read in a small piece of a file or iterate through smaller chunks of the file.

In [ ]:
# Read a large file with read_csv
result = pd.read_csv('examples/ex6.csv')
result

In [ ]:
# Read a small number of rows with nrows
pd.read_csv('examples/ex6.csv', nrows=3)

In [ ]:
# To read a file in pieces, specify a chunksize as a number of rows
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker

# The TextParser object returned by read_csv allows you to iterate over the parts of 
# the file according to the chunksize.

In [ ]:
# Iterate over ex6.csv, aggregating the value counts in the 'key' column
tot = pd.Series([], dtype = 'float64')
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
    
tot = tot.sort_values(ascending=False)
tot[:10]

### Writing Data to Text Format
- Data can also be exported to a delimited format.

In [ ]:
# Read a csv file
data = pd.read_csv('examples/ex5.csv')
data

In [ ]:
# Write data to a csv file using DataFrame’s to_csv method
data.to_csv('examples/output/out.csv')

In [ ]:
# Other delimiters can be used
import sys

data.to_csv(sys.stdout, sep='|')

In [ ]:
# Choose another sentinel value for missing values
data.to_csv(sys.stdout, na_rep='NULL')

In [ ]:
# Disable row and column labels in the output
data.to_csv(sys.stdout, index=False, header=False)

In [ ]:
# Write only a subset of columns
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

In [ ]:
# Series also have a to_csv method
dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('examples/output/tseries.csv')

### Working with Delimited Formats
- Most forms of tabular data from disk using functions like **pandas.read_table**.
- However, some manual processing may be necessary in some cases.
- For any file with a single-character delimiter, you can use Python’s built-in csv module.

In [ ]:
# Example of malformed lines
list(open('examples/ex7.csv'))

In [ ]:
# Import Python’s built-in csv module
import csv

# You need to open the file before passing it to csv.reader
f = open('examples/ex7.csv')
reader = csv.reader(f)

In [ ]:
# Iterating through the reader like a file yields tuples of values with 
# any quote characters removed
for line in reader:
    print(line)

In [ ]:
# First, we read the file into a list of lines
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))
    
lines

In [ ]:
# Split the lines into the header line and the data lines
header, values = lines[0], lines[1:]

In [ ]:
# Create a dictionary of data columns using a dictionary comprehension
# and the expression zip(*values), which transposes rows to columns
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

In [ ]:
# Construct DataFrame from dict
pd.DataFrame.from_dict(data_dict)

## JSON Data
- **JSON** (short for JavaScript Object Notation) has become one of the standard formats for sending data by HTTP request between web browsers and other applications.
- It is a much more free-form data format than a tabular text form like CSV.
- There are several Python libraries for reading and writing JSON data.

In [ ]:
# Define a JSON object
obj = """
{"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
{"name": "Katie", "age": 38,
"pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [ ]:
import json

# Convert a JSON string with json.loads
result = json.loads(obj)
result

In [ ]:
# json.dumps converts a Python object back to JSON
asjson = json.dumps(result)

In [ ]:
# You can pass a list of dicts (which were previously JSON objects) 
# to the DataFrame constructor and select a subset of the data fields

siblings = pd.DataFrame(result['siblings'], columns=['name', 'age', 'pets'])
siblings

In [ ]:
# The pandas.read_json can automatically convert JSON datasets in specific 
# arrangements into a Series or DataFrame

data = pd.read_json('examples/example.json')
data

In [ ]:
# To export data from pandas to JSON the to_json methods on Series and DataFrame
print(data.to_json())

## XML and HTML: Web Scraping
- Python has many libraries for reading and writing data in the ubiquitous HTML and XML formats: **lxml**, **Beautiful Soup**, and **html5lib**.
- While **lxml** is comparatively much faster in general, the other libraries can better handle malformed HTML or XML files.
- **pandas** has a built-in function, **read_html**, which uses libraries like **lxml** and **Beautiful Soup** to automatically parse tables out of HTML files as DataFrame objects.
- The **pandas.read_html** function has a number of options, but by default it searches for and attempts to parse all tabular data contained within table tags.

In [ ]:
# Read a html file with pd.read_html function
tables = pd.read_html('examples/fdic_failed_bank_list.html')

# Check the length of tables and its type
print(type(tables), 'length:', len(tables))

In [ ]:
# Select the only element of the list tables
failures = tables[0]
failures.head()

### Parsing XML with lxml.objectify
- **XML** (eXtensible Markup Language) is another common structured data format supporting hierarchical, nested data with metadata.
- **XML** and **HTML** are structurally similar, but XML is more general.

**XML Example**: 
- The New York Metropolitan Transportation Authority (MTA) publishes a number of data series about its bus and train services.
- Here we’ll look at the performance data, which is contained in a set of XML files. 
- Each train or bus service has a different file (like *Performance_MNR.xml* for the Metro-North Railroad) containing monthly data as a series of XML records.

In [ ]:
# Using lxml.objectify, we parse the file and get a reference to the root node of the
# XML file with getroot

from lxml import objectify

path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [ ]:
# root.INDICATOR returns a generator yielding each <INDICATOR> XML element
# For each record, we can populate a dict of tag names (like YTD_ACTUAL) to data values
# (excluding a few tags)

# Define an empty list
data = []

# Create a list with the tags we want to exclude
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

# Iterate over root.INDICATOR to create a list of dictionaries
for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [ ]:
# Convert this list of dicts into a DataFrame
perf = pd.DataFrame(data)
perf.head()

## Binary Data Formats
- One of the easiest ways to store data (also known as serialization) efficiently is in **binary format**.
- Python has the built-in **pickle** serialization.
- pandas objects all have a **to_pickle** method that writes the data to disk in pickle format.
- **pickle** is only recommended as a short-term storage format. The problem is that it is hard to guarantee that the format will be stable over time.

In [ ]:
# Read data from csv file
frame = pd.read_csv('examples/ex1.csv')
frame

In [ ]:
# Save data into binary format
frame.to_pickle('examples/output/frame_pickle')

In [ ]:
# Read a “pickled” object stored in a file with pandas.read_pickle
pd.read_pickle('examples/output/frame_pickle')

- **pandas** has built-in support for two more binary data formats: **HDF5** and **Message‐Pack**.
- And for **bcolz** =  A compressable column-oriented binary format based on the Blosc compression library.
- And for **Feather** = A cross-language column-oriented file format from the R programming community. Feather uses the Apache Arrow columnar memory format.

### Using HDF5 Format
- **HDF** in **HDF5** stands for hierarchical data format.
- It is available as a C library, and it has interfaces available in many other languages, including Java, Julia, MATLAB, and Python.
- Each **HDF5** file can store multiple datasets and supporting metadata.
- Compared with simpler formats, **HDF5** supports on-the-fly compression with a variety of compression modes, enabling data with repeated patterns to be stored more efficiently.
- HDF5 can be a good choice for working with very large datasets that don’t fit into memory, as you can efficiently read and write small sections of much larger arrays.
- While it’s possible to directly access **HDF5** files using either the **PyTables** or **h5py** libraries, **pandas** provides a high-level interface that simplifies storing Series and
DataFrame object. 
- The **HDFStore** class works like a dict and handles the low-level details.
- If you are processing data that is stored on remote servers, likeAmazon S3 or HDFS, using a different binary format designed for distributed storage like **Apache Parquet** may be more suitable.
- HDF5 is not a database. It is best suited for write-once, read-many datasets. While data can be added to a file at any time, if multiple writers do so simultaneously, the file can become corrupted.

In [ ]:
# Create DataFrame
frame = pd.DataFrame({'a': np.random.randn(100)})
frame

In [ ]:
# Create an HDF5 file
store = pd.HDFStore('mydata.h5')

# Store the frame DataFrame in the HDF5 file 
store['obj1'] = frame

# Store a Series in the HDF5 file 
store['obj1_col'] = frame['a']

# Store the failures DataFrame in the HDF5 file
store['obj2'] = failures

In [ ]:
# Print details about the HDF5 file we created
print(store.info())

In [ ]:
# Objects contained in the HDF5 file can then be retrieved with the same dict-like API
store['obj2']

In [ ]:
# HDFStore supports two storage schemas, 'fixed' and 'table' 
# 'table'  is generally slower, but it supports query operations using a special syntax

store.put('obj2', frame, format='table')

In [ ]:
# Select rows from obj2 stored in 'mydata.h5'
store.select('obj2', where=['index >= 10 and index <= 15'])

In [ ]:
# Only when you run the close() method the data is written to disk
store.close()

In [ ]:
# pandas.read_hdf function gives you a shortcut
pd.read_hdf('mydata.h5', 'obj2', where=['index < 5'])

## Reading Microsoft Excel Files
- **pandas** also supports reading tabular data stored in Excel 2003 (and higher) files using either the ExcelFile class or **pandas.read_excel** function. 
- Internally these tools use the add-on packages **xlrd** and **openpyxl** to read XLS and XLSX files, respectively.

In [ ]:
# Create an instance by passing a path to an xls or xlsx file
xlsx = pd.ExcelFile('examples/ex1.xlsx')

In [ ]:
# Data stored in a sheet can then be read into DataFrame with parse
pd.read_excel(xlsx, 'Sheet1')

In [ ]:
# You can also simply pass the filename to pandas.read_excel
# But if you are reading multiple sheets in a file, then it is faster to create the ExcelFile

frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

In [ ]:
# To write pandas data to Excel format, you must first create an ExcelWriter, 
# then write data to it using pandas objects’ to_excel method

writer = pd.ExcelWriter('examples/output/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

In [ ]:
# You can also pass a file path to to_excel and avoid the ExcelWriter
frame.to_excel('examples/output/ex2.xlsx')

## Interacting with Web APIs
- Many websites have **public APIs** providing data feeds via JSON or some other format.
- An easy way to access these APIs from Python is via the requests package.

**EXAMPLE:** - find the last 30 GitHub issues for pandas on GitHub.

In [ ]:
# Import necessary libraries
import requests

In [ ]:
# Define the url from where to get the data
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'

# Get the data via requests.get method
resp = requests.get(url)
resp

In [ ]:
# The Response object’s json method will return a dictionary containing JSON parsed
# into native Python objects

data = resp.json()

In [ ]:
# Each element in data is a dictionary containing all of the data found on a GitHub
# issue page (except for the comments)

# Get the title for the first issue
data[0]['title']

In [ ]:
# We can pass data directly to DataFrame and extract fields of interest
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])

issues

## Interacting with Databases
- In a business setting, most data may not be stored in text or Excel files. 
- **SQL-based** relational databases (such as SQL Server, PostgreSQL, and MySQL) are in wide use.
- Loading data from SQL into a DataFrame is fairly straightforward, and pandas has some functions to simplify the process.

**EXAMPLE:** - create a SQLite database using Python’s built-in sqlite3 driver.

In [ ]:
# Import neccessary libraries
import sqlite3

In [ ]:
# Create an empty SQL table
query = """
    CREATE TABLE test
    (a VARCHAR(20), b VARCHAR(20),
     c REAL, d INTEGER
    );"""

In [ ]:
# Define the connection to the database
con = sqlite3.connect('mydata.sqlite')

# Execute the query
con.execute(query)

# Commit the changes to the database
con.commit()

In [ ]:
# Insert a few rows of data into test table
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)

con.commit()

In [ ]:
# Most Python SQL drivers (PyODBC, psycopg2, MySQLdb, pymssql, etc.) return a list
# of tuples when selecting data from a table
cursor = con.execute('select * from test')

# Fetch all rows
rows = cursor.fetchall()

In [ ]:
# Column names are contained in the cursor’s description attribute
cursor.description

In [ ]:
# You can pass the list of tuples to the DataFrame constructor + the columns names
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

### SQLAlchemy
- The **SQLAlchemy** project is a popular Python SQL toolkit that abstracts away many of the common differences between SQL databases. 
- **pandas** has a **read_sql** function that enables you to read data easily from a general SQLAlchemy connection.
- Here, we’ll connect to the same SQLite database with SQLAlchemy and read data from the table created before.

In [ ]:
# Import the package
import sqlalchemy as sqla

In [ ]:
# Create teh sqla engine
db = sqla.create_engine('sqlite:///mydata.sqlite')

In [ ]:
# Use pandas read_sql function to read the data directly into a DataFrame
pd.read_sql('select * from test', db)